## Sim Demo

### Imports

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from env.imports import *

  import numpy.matlib as npm

  autumn = cm.get_cmap('autumn', 256)

  winter = cm.get_cmap('winter_r', 256)



In [4]:
import importlib
import data

import models
import sim.sim
import sim.sim_utils
from sim.sim_utils import bytes2human, print_system_usage
from sim.sim import Simulation
from sim.sim_run import single_sim_run

importlib.reload(sim.sim)
importlib.reload(sim.sim_run)
importlib.reload(models.train_val)
importlib.reload(data.data_utils)


<module 'data.data_utils' from '/scratch/asr655/neuroinformatics/GeneEx2Conn/data/data_utils.py'>

#### Check job specs

In [5]:
print_system_usage()

total = psutil.disk_usage('/').total
print(bytes2human(total))

CPU Usage: 2.5%
RAM Usage: 3.9%
Available RAM: 1.4T
Total RAM: 1.5T
52.4G


In [6]:
print(os.environ.get("CUDA_VISIBLE_DEVICES"))

# Check available GPUs
print(f"Available GPUs: {torch.cuda.device_count()}")

for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)} - Memory Allocated: {torch.cuda.memory_allocated(i)/1024**3:.2f} GB")


0
Available GPUs: 1
GPU 0: NVIDIA H100 80GB HBM3 - Memory Allocated: 0.00 GB


In [7]:
print("XGBoost version:", xgboost.__version__)
print("cupy version:", cp.__version__)

GPUtil.getAvailable()

# if a number is seen a GPU is available
GPUtil.getGPUs()

DEVICE_ID_LIST = GPUtil.getFirstAvailable()
DEVICE_ID = DEVICE_ID_LIST[0] # grab first element from list
if DEVICE_ID != None: 
    print('GPU found', DEVICE_ID)
    use_gpu = True

    GPUtil.showUtilization()

XGBoost version: 2.0.3
cupy version: 13.1.0
GPU found 0
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [8]:
torch.cuda.is_available()

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

## Simulation tests <a id="sims"></a>

### Model Parameter Counts

#### **Linear Models**
- **PCA Bilinear:** 730  _(27 PCs)_
- **PLS Bilinear:** 101  _(10 PLS components)_
- **Bilinear Low-rank:** 73,800  _(rank 10)_
- **PLS MLP:** 158,993  _(10 PLS components, including PLS projection matrices)_
- **PCA MLP:** 47,873  _(27 PCs, 2-layer)_

---

#### **MLP and SMT Models**

#### 2-Layer Models
- **MLP:** 3,812,609
- **SMT:** 1,399,947
- **MLP w/ CLS:** 3,814,145
- **SMT w/ CLS:** 1,405,579

#### 3-Layer Models
- **MLP:** 7,723,777
- **SMT:** 2,162,315
- **MLP w/ CLS:** 7,726,849
- **SMT w/ CLS:** 2,173,067

---

### Coord MLP Parameter Counts
- **[32]:** 321
- **[64, 32]:** 2,753
- **[128, 64]:** 9,601
- **[256, 128]:** 35,685
- **[512, 256, 128]:** 169,729

In [9]:
torch._dynamo.reset()

In [ ]:
    single_sim_run(
                  feature_type=[{'transcriptome': None}],
                  train_shared_regions=False,
                  test_shared_regions=False,
                  omit_subcortical=False,
                  dataset='UKBB',
                  parcellation='S456',
                  hemisphere='both',
                  connectome_target='FC',
                  binarize=None,
                  impute_strategy='mirror_interpolate',
                  sort_genes='refgenome',
                  gene_list='0.2',
                  cv_type='random',
                  random_seed=42,
                  search_method=('wandb', 'mse', 5),
                  track_wandb=True,
                  skip_cv=False,
                  model_type='shared_transformer',
                  use_gpu=True,
                  null_model='none',
                  use_folds=[0]
                  )
    
    # Clear GPU memory
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # Clear CPU memory
    gc.collect()

Number of components for 95% variance PCA: 27
Number of 1s: 36662, Number of 0s: 171274, Class balance (1s): 0.176
Number of 1s: 17738, Number of 0s: 190198, Class balance (1s): 0.085
X shape: (455, 7380)
X_pca shape: (455, 27)
X_cell_types_Jorstad shape: (455, 24)
X_cell_types_LakeDFC shape: (455, 18)
X_cell_types_LakeVIS shape: (455, 18)
Y_sc shape: (455, 455)
Y_sc_spectralL shape: (455, 455)
Y_sc_spectralA shape: (455, 456)
Y_fc shape: (455, 455)
Coordinates shape: (455, 3)
Y shape (455, 455)
feature_name: transcriptome, processing_type: None
features ['transcriptome']
Feature matrix, X, generated... expanding to pairwise dataset


  return LooseVersion(v) >= LooseVersion(check)

ERROR: Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: asratzan (alexander-ratzan-new-york-university). Use `wandb login --relogin` to force relogin
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Additional properties are not allowed ('best_parameters' was unexpected)


Create sweep with ID: 2ijbq53r
Sweep URL: https://wandb.ai/alexander-ratzan-new-york-university/gx2conn/sweeps/2ijbq53r
Initialized sweep with ID: 2ijbq53r
2
3
4


wandb: Agent Starting Run: hd8les5s with config:
wandb: 	aug_prob: 0.3
wandb: 	batch_size: 1024
wandb: 	d_model: 128
wandb: 	deep_hidden_dims: [512, 256, 128]
wandb: 	dropout_rate: 0.2
wandb: 	encoder_output_dim: 10
wandb: 	epochs: 100
wandb: 	input_dim: 14760
wandb: 	learning_rate: 5e-05
wandb: 	nhead: 4
wandb: 	num_layers: 4
wandb: 	num_workers: 2
wandb: 	prefetch_factor: 4
wandb: 	token_encoder_dim: 60
wandb: 	transformer_dropout: 0.2
wandb: 	use_alibi: True
wandb: 	use_attention_pooling: True
wandb: 	weight_decay: 1e-05
ERROR: Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
  self._sweep_config = config_util.dict_from_config_file(

  self.comm = Comm(**args)

wandb: WARNING Ignored wandb.init() arg project when running a sweep.

  from IPython.core.display import HTML, display  # type: ignore



Processing inner fold 0


...

BEST CONFIG {'nhead': 4, 'epochs': 100, 'd_model': 128, 'aug_prob': 0.3, 'input_dim': 14760, 'use_alibi': True, 'batch_size': 1024, 'num_layers': 4, 'num_workers': 2, 'dropout_rate': 0.2, 'weight_decay': 1e-05, 'learning_rate': 0.0003, 'prefetch_factor': 4, 'deep_hidden_dims': [512, 256, 128], 'token_encoder_dim': 60, 'encoder_output_dim': 10, 'transformer_dropout': 0.2, 'use_attention_pooling': True}
Number of learnable parameters in SMT model: 1036802


  self.comm = Comm(**args)

  from IPython.core.display import HTML, display  # type: ignore



Using device: cuda
GPU 0: NVIDIA H100 80GB HBM3 - Memory Allocated: 0.07 GB


  item = pop()



Best val loss so far at epoch 1: 0.0387


In [ ]:
single_sim_run(
              feature_type=[{'transcriptome': None}],
              train_shared_regions=False,
              test_shared_regions=False,
              omit_subcortical=False,
              dataset='UKBB',
              parcellation='S456',
              hemisphere='both',
              connectome_target='FC',
              binarize=None,
              impute_strategy='mirror_interpolate',
              sort_genes='refgenome',
              gene_list='0.2',
              cv_type='random',
              random_seed=42,
              search_method=('wandb', 'mse', 4),
              track_wandb=False,
              skip_cv=True,
              model_type='shared_transformer_cls',
              use_gpu=True,
              null_model='none',
              use_folds=[0]
              )

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Clear CPU memory
gc.collect()